In [ ]:
!nvidia-smi

Mon Nov 20 17:21:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Dependences

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
from dataclasses import dataclass
import torch
from typing import Generator, Tuple, Optional, List, Dict, Any
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm.notebook import tqdm
from google.colab import files
from pathlib import Path
%matplotlib inline

##Detectors & Trackers

###YOLOv5

In [ ]:
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt
import utils
display = utils.notebook_init()
%cd {HOME}

YOLOv5 🚀 v7.0-243-g7c54e5d Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)
/content


###YOLOv8

In [ ]:
!pip install supervision
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 1.6 MB/s eta 0:00:00


### ByteTrack

In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)
%cd {HOME}

yolox.__version__: 0.1.0
/content


###StrongSORT

In [ ]:
%cd {HOME}
!git clone https://github.com/mikel-brostrom/yolo_tracking.git
%cd {HOME}/yolo_tracking

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop

sys.path.append(f"{HOME}/yolo_tracking")
%cd {HOME}

###SORT

In [ ]:
!apt install libopencv-dev
!apt install libeigen3-dev
!pip install sort-tracker
import sort

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libeigen3-doc libmpfrc++-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 1,056 kB of archives.
After this operation, 9,081 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeigen3-dev all 3.4.0-2ubuntu2 [1,056 kB]
Fetched 1,056 kB in 1s (1,071 kB/s)
Selecting previously unselected package libeigen3-dev.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../libeigen3-dev_3.4.0-2ubuntu2_all.deb ...
Unpacking libeigen3-dev (3.4.0-2ubuntu2) ...
Setting up libeige

##Weifhts & Videos

In [ ]:
!gdown 1JJQb8IGv5NrhRiTFabrjFULwSrDMbmMs
!unzip TEST.zip

Downloading...
From: https://drive.google.com/uc?id=1JJQb8IGv5NrhRiTFabrjFULwSrDMbmMs
To: /content/TEST.zip
100% 1.52G/1.52G [00:16<00:00, 90.2MB/s]
Archive:  TEST.zip
   creating: TEST/
   creating: TEST/test1/
  inflating: TEST/test1/video.avi    
  inflating: TEST/test1/YOLOv5.pt    
  inflating: TEST/test1/YOLOv8.pt    
  inflating: TEST/test1/labels.txt   
  inflating: TEST/test1/background.png  
   creating: TEST/test2/
  inflating: TEST/test2/video.avi    
  inflating: TEST/test2/YOLOv5.pt    
  inflating: TEST/test2/YOLOv8.pt    
  inflating: TEST/test2/labels.txt   
  inflating: TEST/test2/background.png  
   creating: TEST/test3/
  inflating: TEST/test3/labels.txt   
  inflating: TEST/test3/video.avi    
  inflating: TEST/test3/YOLOv5.pt    
  inflating: TEST/test3/YOLOv8.pt    
  inflating: TEST/test3/background.png  
   creating: TEST/test4/
  inflating: TEST/test4/labels.txt   
  inflating: TEST/test4/video.avi    
  inflating: TEST/test4/YOLOv5.pt    
  inflating: TEST/te

#Config

##Base Classes

### Geometry utilities

In [ ]:
from __future__ import annotations

@dataclass(frozen=True)
class Point:
    x: float
    y: float

    @property
    def int_xy_tuple(self) -> Tuple[int, int]:
        return int(self.x), int(self.y)


@dataclass(frozen=True)
class Rect:
    x: float
    y: float
    width: float
    height: float

    @property
    def min_x(self) -> float:
        return self.x

    @property
    def min_y(self) -> float:
        return self.y

    @property
    def s_width(self) -> float:
        return self.x + self.width

    @property
    def s_height(self) -> float:
        return self.y + self.height

    @property
    def max_x(self) -> float:
        return self.x + self.width

    @property
    def max_y(self) -> float:
        return self.y + self.height

    @property
    def top_left(self) -> Point:
        return Point(x=self.x, y=self.y)

    @property
    def bottom_right(self) -> Point:
        return Point(x=self.x + self.width, y=self.y + self.height)

    @property
    def bottom_center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y + self.height)

    @property
    def top_center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y)

    @property
    def center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y + self.height / 2)

    def get_bottom_center(self) -> tuple:
      return (int(self.x + self.width / 2), int(self.y + self.height))

    def get_center(self) -> tuple:
      return (int(self.x + self.width / 2), int(self.y + self.height / 2))

    def min_max_cords(self):

      return self.min_x, self.min_y, self.max_x, self.max_y


    def pad(self, padding: float) -> Rect:
        return Rect(
            x=self.x - padding,
            y=self.y - padding,
            width=self.width + 2*padding,
            height=self.height + 2*padding
        )


    def contains_point(self, point: Point) -> bool:
        return self.min_x < point.x < self.max_x and self.min_y < point.y < self.max_y

### Detection utilities

In [ ]:
@dataclass
class Detection:
    rect: Rect
    class_id: int
    class_name: str
    confidence: float
    tracker_id: Optional[int] = None

    @classmethod
    def from_results(cls, pred: np.ndarray, names: Dict[int, str]) -> List[Detection]:
        result = []
        for x_min, y_min, x_max, y_max, confidence, class_id in pred:
            class_id=int(class_id)
            result.append(Detection(
                rect=Rect(
                    x=float(x_min),
                    y=float(y_min),
                    width=float(x_max - x_min),
                    height=float(y_max - y_min)
                ),
                class_id=class_id,
                class_name=names[class_id],
                confidence=float(confidence)
            ))
        return result

    def available_for_annotation(self):
      if self.class_id != None and self.class_name != None and self.tracker_id != None:
        print(self.tracker_id)
        return True
      else:
        return False

def filter_detections_by_class(detections: List[Detection], class_name: str) -> List[Detection]:
    return [
        detection
        for detection
        in detections
        if detection.class_name == class_name
    ]

### Draw utilities

In [ ]:
@dataclass(frozen=True)
class Color:
    r: int
    g: int
    b: int

    @property
    def bgr_tuple(self) -> Tuple[int, int, int]:
        return self.b, self.g, self.r

    @classmethod
    def from_hex_string(cls, hex_string: str) -> Color:
        r, g, b = tuple(int(hex_string[1 + i:1 + i + 2], 16) for i in (0, 2, 4))
        return Color(r=r, g=g, b=b)


def draw_rect(image: np.ndarray, rect: Rect, color: Color, thickness: int = 2) -> np.ndarray:
    cv2.rectangle(image, rect.top_left.int_xy_tuple, rect.bottom_right.int_xy_tuple, color.bgr_tuple, thickness)
    return image


def draw_filled_rect(image: np.ndarray, rect: Rect, color: Color) -> np.ndarray:
    cv2.rectangle(image, rect.top_left.int_xy_tuple, rect.bottom_right.int_xy_tuple, color.bgr_tuple, -1)
    return image


def draw_polygon(image: np.ndarray, countour: np.ndarray, color: Color, thickness: int = 2) -> np.ndarray:
    cv2.drawContours(image, [countour], 0, color.bgr_tuple, thickness)
    return image


def draw_filled_polygon(image: np.ndarray, countour: np.ndarray, color: Color) -> np.ndarray:
    cv2.drawContours(image, [countour], 0, color.bgr_tuple, -1)
    return image


def draw_text(image: np.ndarray, anchor: Point, text: str, color: Color, thickness: int = 2) -> np.ndarray:
    cv2.putText(image, text, anchor.int_xy_tuple, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color.bgr_tuple, thickness, 2, False)
    return image

### Annotation utilities

In [ ]:
@dataclass
class BaseAnnotator:
    colors: List[Color]
    thickness: int

    def annotate(self, image: np.ndarray, detections: List[Detection]) -> np.ndarray:
        annotated_image = image.copy()
        for detection in detections:
            annotated_image = draw_rect(
                image=image,
                rect=detection.rect,
                color=self.colors[detection.class_id],
                thickness=self.thickness
            )
        return annotated_image

# calculates coordinates of possession marker
def calculate_marker(anchor: Point) -> np.ndarray:
    x, y = anchor.int_xy_tuple
    return(np.array([
        [x - MARKER_WIDTH // 2, y - MARKER_HEIGHT - MARKER_MARGIN],
        [x, y - MARKER_MARGIN],
        [x + MARKER_WIDTH // 2, y - MARKER_HEIGHT - MARKER_MARGIN]
    ]))


# draw single possession marker
def draw_marker(image: np.ndarray, anchor: Point, color: Color) -> np.ndarray:
    possession_marker_countour = calculate_marker(anchor=anchor)
    image = draw_filled_polygon(
        image=image,
        countour=possession_marker_countour,
        color=color)
    image = draw_polygon(
        image=image,
        countour=possession_marker_countour,
        color=MARKER_CONTOUR_COLOR,
        thickness=MARKER_CONTOUR_THICKNESS)
    return image


# dedicated annotator to draw possession markers on video frames
@dataclass
class MarkerAnntator:

    color: Color

    def annotate(self, image: np.ndarray, detections: List[Detection]) -> np.ndarray:
        annotated_image = image.copy()
        for detection in detections:
            annotated_image = draw_marker(
                image=image,
                anchor=detection.rect.top_center,
                color=self.color)
        return annotated_image

@dataclass
class ObjectAnnotator:

    color: Color

    def circle(self, image: np.ndarray, detections: List[Detection]) -> np.ndarray:
        annotated_image = image.copy()
        for detection in detections:
            annotated_image = cv2.circle(annotated_image, detection.rect.get_bottom_center(), 2*int(detection.rect.width), self.color.bgr_tuple, 2)
        return annotated_image

    def line(self, image: np.ndarray, object_detections: List[Detection], person_detections: List[Detection]) -> np.ndarray:
        annotated_image = image.copy()
        for person_detection in person_detections:
            person_center= person_detection.rect.get_center()
            for object_detection in object_detections:
                object_center = object_detection.rect.get_center()
                annotated_image = cv2.line(annotated_image,person_center,object_center,self.color.bgr_tuple,2)
        return annotated_image

# stores information about output video file, width and height of the frame must be equal to input video
@dataclass(frozen=True)
class VideoConfig:
    fps: float
    width: int
    height: int


# create cv2.VideoWriter object that we can use to save output video
def get_video_writer(target_video_path: str, video_config: VideoConfig) -> cv2.VideoWriter:
    video_target_dir = os.path.dirname(os.path.abspath(target_video_path))
    os.makedirs(video_target_dir, exist_ok=True)
    return cv2.VideoWriter(
        target_video_path,
        fourcc=cv2.VideoWriter_fourcc(*"mp4v"),
        fps=video_config.fps,
        frameSize=(video_config.width, video_config.height),
        isColor=True
    )

# text annotator to display tracker_id
@dataclass
class TextAnnotator:
    background_color: Color
    text_color: Color
    text_thickness: int

    def annotate(self, image: np.ndarray, detections: List[Detection]) -> np.ndarray:
        annotated_image = image.copy()
        for detection in detections:
            # if tracker_id is not assigned skip annotation
            if detection.tracker_id is None:
                continue

            # calculate text dimensions
            size, _ = cv2.getTextSize(
                str(detection.tracker_id),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                thickness=self.text_thickness)
            width, height = size

            # calculate text background position
            center_x, center_y = detection.rect.bottom_center.int_xy_tuple
            x = center_x - width // 2
            y = center_y - height // 2 + 10

            # draw background
            annotated_image = draw_filled_rect(
                image=annotated_image,
                rect=Rect(x=x, y=y, width=width, height=height).pad(padding=5),
                color=self.background_color)

            # draw text
            annotated_image = draw_text(
                image=annotated_image,
                anchor=Point(x=x, y=y + height),
                text=str(detection.tracker_id),
                color=self.text_color,
                thickness=self.text_thickness)
        return annotated_image

In [ ]:
def generate_frames(video_file: str) -> Generator[np.ndarray, None, None]:
    video = cv2.VideoCapture(video_file)

    while video.isOpened():
        success, frame = video.read()

        if not success:
            break

        yield frame

    video.release()

def plot_image(image: np.ndarray, size: int = 12) -> None:
    plt.figure(figsize=(size, size))
    plt.imshow(image[...,::-1])
    plt.show()

## Regular objects detection methods

In [ ]:
def ellipse_check(center,point,rad,coef):
    first= (coef[0]*(point[0]-center[0])**2 + coef[1]*((point[0]-center[0])*(point[1]-center[1])) + coef[2]*(point[1]-center[1])**2)
    rad=rad**2
    if( first <= rad):
        return True
    else:
        return False

def ellipse_check2(center,point,rad,coef):
    first= (coef[0]*(point[0]-center[0])**2 + coef[1]*((point[0]-center[0])*(point[1]-center[1])) + coef[2]*(point[1]-center[1])**2)
    rad=rad**2
    if( first >= (0.9 * rad) and first <= (1.1*rad)):
        return True
    else:
        return False

def draw_the_ellipse(center, image,radius, coef):
    h, w, c= image.shape
    image = cv2.circle(image, center, radius=1, color=(99, 112, 222), thickness=-1)
    for x in range(w):
        for y in range(h):
            if(x % 4 == 0 and y % 4 == 0 and ellipse_check2(center, (x,y), radius,coef)) :
                image = cv2.circle(image, (x, y), radius=2, color=(0, 0, 255), thickness=-1)

    return image

def draw_elipce_around_abandoned_object(
    object_detections: List[Detection],
    image, ellipse_ratio
):
  for object_detection in object_detections:

    object_coordinates = object_detection.rect.get_bottom_center()
    object_width= int(object_detection.rect.width)

    image = draw_the_ellipse(object_coordinates, image, 35*object_width, ellipse_ratio)

  return image

### Point in a circle

A method of searching for regular objects based on finding at least one point of a person's box inside a circle


In [ ]:
def get_regular_object(
    person_detections: List[Detection],
    object_detections: List[Detection]
) -> Optional[Detection]:

  regular=[]
  for object_detection in object_detections:
    for person_detection in person_detections:
      if object_detection.rect.pad(2*int(object_detection.rect.width)).contains_point(point=person_detection.rect.bottom_center):
        regular.append(object_detection)
  return regular

In [ ]:
# def get_regular_object(
#     person_detections: List[Detection],
#     object_detections: List[Detection],
#     image
# ) -> Optional[Detection]:

#   regular=[]
#   for object_detection in object_detections:
#     for person_detection in person_detections:
#       if object_detection.rect.pad(2*int(object_detection.rect.width)).contains_point(point=person_detection.rect.bottom_center):
#         person_center= person_detection.rect.get_center()
#         print(object_detection.rect.get_bottom_center())
#         regular.append(object_detection)
#         image = cv2.circle(image, object_detection.rect.get_bottom_center(), 2*int(object_detection.rect.width), (0,255,0), 2)
#       else:
#         print(object_detection.rect.get_bottom_center())
#         image = cv2.circle(image, object_detection.rect.get_bottom_center(), 2*int(object_detection.rect.width), (0,0,255), 2)
#   return regular, image

### A point on an ellipse

A method of searching for ordinary objects based on finding the middle of the lower face of a person's box inside an ellipse, the size of which depends on the width of the box of the object

In [ ]:
def get_regular_object_second(
    person_detections: List[Detection],
    object_detections: List[Detection],
    ellipse_ratio
) -> Optional[Detection]:
  regular=[]
  for object_detection in object_detections:
    for person_detection in person_detections:

      object_coordinates = object_detection.rect.get_bottom_center()
      person_coordinates = person_detection.rect.get_bottom_center()
      object_width= int(object_detection.rect.width)

      if (ellipse_check(object_coordinates, person_coordinates, 50*object_width, ellipse_ratio)):
        regular.append(object_detection)

  return regular

### A point in an ellipse

A method of searching for regular objects based on finding at least one point of a person's box inside an ellipse, the size of which
depends on the width of the box of the object

In [ ]:
def get_regular_object_third(
    person_detections: List[Detection],
    object_detections: List[Detection],
    ellipse_ratio
) -> Optional[Detection]:
  regular=[]
  for object_detection in object_detections:
    for person_detection in person_detections:
      object_coordinates = object_detection.rect.get_bottom_center()
      object_width= int(object_detection.rect.width)
      min_x, min_y, max_x, max_y= person_detection.rect.min_max_cords()
      loop_breaker=0

      for y in range(int(min_y), int(max_y)):
        for x in range(int(min_x), int(max_x)):
          if (ellipse_check(object_coordinates, (x,y), 35*object_width, ellipse_ratio)):
            regular.append(object_detection)
            loop_breaker=1
            break
        if loop_breaker: break
  return regular

### Euclidean distance

The method of searching for regular objects based on the Euclidean distance between the centers of the boxes
of a person and an object, the distance depends on the width of the box of the object.

In [ ]:
from scipy.spatial import distance

def get_regular_object_fourth(
    person_detections: List[Detection],
    object_detections: List[Detection]
) -> Optional[Detection]:
  regular=[]
  for object_detection in object_detections:
    for person_detection in person_detections:

      object_center = object_detection.rect.get_center()
      person_center= person_detection.rect.get_center()

      object_width= int(object_detection.rect.width)
      dst = distance.euclidean(object_center, person_center)

      if (dst <= 2.5*object_width):
        regular.append(object_detection)
  return regular#, image

In [ ]:
# from scipy.spatial import distance

# def get_regular_object_fourth(
#     person_detections: List[Detection],
#     object_detections: List[Detection],
#     image
# ) -> Optional[Detection]:
#   regular=[]
#   for object_detection in object_detections:
#     for person_detection in person_detections:

#       object_center = object_detection.rect.get_center()
#       person_center= person_detection.rect.get_center()

#       object_width= int(object_detection.rect.width)
#       dst = distance.euclidean(object_center, person_center)

#       if (dst <= 2.5*object_width):
#         regular.append(object_detection)
#         image = cv2.line(image,person_center,object_center,(0,255,0),2)
#       else:
#         image = cv2.line(image,person_center,object_center,(0,0,255),2)
#   return regular, image

###Chebyshev distance

The method of searching for ordinary objects based on the Chebyshev distance between the centers of the boxes
of a person and an object, the distance depends on the width of the box of the object.

In [ ]:
from scipy.spatial import distance

def get_regular_object_fifth(
    person_detections: List[Detection],
    object_detections: List[Detection]
) -> Optional[Detection]:
  regular=[]
  for object_detection in object_detections:
    for person_detection in person_detections:

      object_center = object_detection.rect.get_center()
      person_center= person_detection.rect.get_center()
      object_width= int(object_detection.rect.width)
      dst = distance.chebyshev(object_center, person_center)

      if (dst <= 2*object_width):
        regular.append(object_detection)
       # cv2.line(image,person_center,object_center,(0,0,255),2)

  return regular#, image

## Identification of abandoned and suspicious objects

In [ ]:
def detections2boxes_for_single_detection(detection):
  return np.array([[
      detection.rect.top_left.x,
      detection.rect.top_left.y,
      detection.rect.bottom_right.x,
      detection.rect.bottom_right.y
      ]], dtype=float)


def classification_of_abandoned_objects(arr_of_abandoned_objects,
                                        det_of_abandobed_objects, # only one det
                                        frame_number):
  removable=[]
  for j in range(len(det_of_abandobed_objects)):
      removable=[]
      check_the_same = 0
      det_info =  np.empty((1,4), dtype=np.object)
      det_info[0][0]= det_of_abandobed_objects[j] # detection
      det_info[0][1]= False # statement
      det_info[0][2]= [] # ab_track
      det_info[0][3]= frame_number # last_frame
      det_box = detections2boxes_for_single_detection(det_info[0][0])
      arr_of_abandoned_objects=np.append(arr_of_abandoned_objects, det_info, axis=0)

      for i in range(len(arr_of_abandoned_objects)):
        delta_frame = arr_of_abandoned_objects[i][3]
        arr_box = detections2boxes_for_single_detection(arr_of_abandoned_objects[i][0])
        if box_iou_batch(arr_box, det_box) > 0.6 and check_the_same!=0:
            arr_of_abandoned_objects[i][2].extend(det_info[0][2])
            removable.append(i)
        if box_iou_batch(arr_box, det_box) > 0.6 and check_the_same==0:
            if det_info[0][0].tracker_id != None:
              arr_of_abandoned_objects[i][0] = det_info[0][0]
            arr_of_abandoned_objects[i][2].append(frame_number)
            arr_of_abandoned_objects[i][2].extend(det_info[0][2])
            arr_of_abandoned_objects[i][3] = det_info[0][3]
            check_the_same = 1
      arr_of_abandoned_objects = np.delete(arr_of_abandoned_objects, removable, 0)
  arr_of_abandoned_objects = arr_of_abandoned_objects[frame_number - arr_of_abandoned_objects[:, 3] < 200] #10 sec in video
  for i in range(len(arr_of_abandoned_objects)):
    arr_of_abandoned_objects[i][2] = list((set(arr_of_abandoned_objects[i][2])))
    if len(arr_of_abandoned_objects[i][2]) > 100:
      ab_t = [x for x in arr_of_abandoned_objects[i][2] if frame_number-x<=200]
      if len(ab_t) > 60:
        arr_of_abandoned_objects[i][1] = True
      ab_t= None

  return arr_of_abandoned_objects

## MOT

In [ ]:
def create_or_empty_file(file_name):
    if os.path.exists(file_name):
        with open(file_name, 'w') as file:
            pass
    else:
        open(file_name, 'w').close()
""" <frame number>, <object id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <confidence>, <x>, <y>, <z> """
def mot_txt(filename_for_mot, frame_number, detections):
    for det in detections:
        string_of_det= str(frame_number)+','+str(det.tracker_id if det.tracker_id!=None else -1)+','\
            +str(round(det.rect.x, 2))+','+str(round(det.rect.y, 2))+','+str(round(det.rect.width, 2))+','\
            +str(round(det.rect.height, 2))+','+str(round(det.confidence, 2))+','+'-1'+','+'-1'+','+'-1'
        with open(filename_for_mot, 'r+') as f:
            f.seek(0, 2)  # перемещение курсора в конец файла
            f.write(f'{string_of_det}\n')  # собственно, запись

#Video Configurations

In [ ]:
video_for_test=2

if video_for_test == 1:
    dir_of_test="/content/TEST/test1"
    video_name="test1"
if video_for_test == 2:
    dir_of_test="/content/TEST/test2"
    video_name="test2"
if video_for_test == 3:
    dir_of_test="/content/TEST/test3"
    video_name="test3"
if video_for_test == 4:
    dir_of_test="/content/TEST/test4"
    video_name="test4"
if video_for_test == 5:
    dir_of_test="/content/TEST/test5"
    video_name="test5"


In [ ]:
SOURCE_VIDEO_PATH = f'{dir_of_test}/video.avi'

vid = cv2.VideoCapture(SOURCE_VIDEO_PATH)
height_of_source = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
width_of_source = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
fps_of_source = int(vid.get(cv2.CAP_PROP_FPS))
length_of_source = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
vid.release()

el_ratio=(260,100,1630)


print(height_of_source)
print(width_of_source)
print(fps_of_source)
print(length_of_source)

414
736
20
977


#Models

##BYTETracker

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch

@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.35
    track_buffer: int = 300
    match_thresh: float = 0.6
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False


# converts List[Detection] into format that can be consumed by match_detections_with_tracks function
def detections2boxes_BYTE(detections: List[Detection], with_confidence: bool = True) -> np.ndarray:
    return np.array([
        [
            detection.rect.top_left.x,
            detection.rect.top_left.y,
            detection.rect.bottom_right.x,
            detection.rect.bottom_right.y,
            detection.confidence
            # detection.class_id #!!!!!
        ] if with_confidence else [
            detection.rect.top_left.x,
            detection.rect.top_left.y,
            detection.rect.bottom_right.x,
            detection.rect.bottom_right.y
        ]
        for detection
        in detections
    ], dtype=float)


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes_BYTE(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)

# matches our bounding boxes with predictions
def match_detections_with_tracks_BYTE(
    detections: List[Detection],
    tracks: List[STrack]
) -> List[Detection]:
    detection_boxes = detections2boxes_BYTE(detections=detections, with_confidence=False)
    tracks_boxes = tracks2boxes_BYTE(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detection_boxes)
    track2detection = np.argmax(iou, axis=1)
    for tracker_index, detection_index in enumerate(track2detection):
        detections[detection_index].tracker_id = tracks[tracker_index].track_id

    return detections

##StrongSORT

In [ ]:
from boxmot.trackers.strongsort.strong_sort import StrongSORT
from onemetric.cv.utils.iou import box_iou_batch

def detections2boxes_STRONG(detections: List[Detection], with_confidence: bool = True) -> np.ndarray:
    return np.array([
        [
            detection.rect.top_left.x,
            detection.rect.top_left.y,
            detection.rect.bottom_right.x,
            detection.rect.bottom_right.y,
            detection.confidence,
            detection.class_id #!!!!!
        ] if with_confidence else [
            detection.rect.top_left.x,
            detection.rect.top_left.y,
            detection.rect.bottom_right.x,
            detection.rect.bottom_right.y
        ]
        for detection
        in detections
    ], dtype=float)


# converts tracks into format that can be consumed by match_detections_with_tracks function
def tracks2boxes_STRONG(tracks):
    format_tracks = np.empty((0,4))
    for i in range(len(tracks)):
      arr_2d = np.reshape(tracks[i][:4], (1, 4))
      format_tracks=np.append(format_tracks, arr_2d, axis=0)
    return format_tracks

# matches our bounding boxes with predictions
def match_detections_with_tracks_STRONG(
    detections: List[Detection],
    tracks
) -> List[Detection]:
    detection_boxes = detections2boxes_STRONG(detections=detections, with_confidence=False)
    tracks_boxes = tracks2boxes_STRONG(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detection_boxes)
    track2detection = np.argmax(iou, axis=1)
    for tracker_index, detection_index in enumerate(track2detection):
        track_id=tracks[tracker_index][4].astype('int')
        detections[detection_index].tracker_id = track_id#
    return detections

##SORT

In [ ]:
from onemetric.cv.utils.iou import box_iou_batch

def detections2boxes_SORT(detections: List[Detection], with_confidence: bool = True) -> np.ndarray:
    return np.array([
        [
            detection.rect.min_x,
            detection.rect.min_y,
            detection.rect.s_width,
            detection.rect.s_height,
            detection.confidence
        ] if with_confidence else [
            detection.rect.min_x,
            detection.rect.min_y,
            detection.rect.s_width,
            detection.rect.s_height
        ]
        for detection
        in detections
    ], dtype=float)


# converts tracks into format that can be consumed by match_detections_with_tracks function
def tracks2boxes_SORT(tracks):
    format_tracks = np.empty((0,4))
    for i in range(len(tracks)):
      arr_2d = np.reshape(tracks[i][1:5], (1, 4))
      format_tracks=np.append(format_tracks, arr_2d, axis=0)
    return format_tracks

# matches our bounding boxes with predictions
def match_detections_with_tracks_SORT(
    detections: List[Detection],
    tracks
) -> List[Detection]:
    detection_boxes = detections2boxes_SORT(detections=detections, with_confidence=False)
    tracks_boxes = tracks2boxes_SORT(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detection_boxes)
    track2detection = np.argmax(iou, axis=1)
    for tracker_index, detection_index in enumerate(track2detection):
        track_id=tracks[tracker_index][4].astype('int')
        detections[detection_index].tracker_id = track_id#
    return detections

##YOLOv5

In [ ]:
def Detections_from_YOLOv5(results):
    return results.pred[0].cpu().numpy()

##YOLOv8

In [ ]:
def Detections_from_YOLOv8(results):
    xyxy=results[0].boxes.xyxy.cpu().numpy()
    conf=results[0].boxes.conf.cpu().numpy()
    class_id=results[0].boxes.cls.cpu().numpy()
    return np.column_stack((xyxy,conf,class_id))

#MAIN

##Annotation Configuration

In [ ]:
MARKER_CONTOUR_COLOR_HEX = "#000000"
MARKER_CONTOUR_COLOR = Color.from_hex_string(MARKER_CONTOUR_COLOR_HEX)

# SpringGreen
REGUAL_OBJECT_FILL_COLOR_HEX = "#00FF7F"
REGUAL_OBJECT_FILL_COLOR = Color.from_hex_string(REGUAL_OBJECT_FILL_COLOR_HEX)

# Dark Red
ABANDONED_OBJECT_FILL_COLOR_HEX = "#8B0000"
ABANDONED_OBJECT_FILL_COLOR = Color.from_hex_string(ABANDONED_OBJECT_FILL_COLOR_HEX)

# Blue
PERSON_FILL_COLOR_HEX = "#0000FF"
PERSON_FILL_COLOR = Color.from_hex_string(PERSON_FILL_COLOR_HEX)

# Deep Oranje
SUSPICIOUS_OBJECT_FILL_COLOR_HEX = "#C34D0A"
SUSPICIOUS_OBJECT_FILL_COLOR = Color.from_hex_string(SUSPICIOUS_OBJECT_FILL_COLOR_HEX)

MARKER_CONTOUR_THICKNESS = 2
MARKER_WIDTH = 18
MARKER_HEIGHT = 18
MARKER_MARGIN = 8

person_text_annotator = TextAnnotator(
    PERSON_FILL_COLOR, text_color=Color(255, 255, 255), text_thickness=2)

regular_object_text_annotator = TextAnnotator(
    REGUAL_OBJECT_FILL_COLOR, text_color=Color(0, 0, 0), text_thickness=2)

abandoned_object_text_annotator = TextAnnotator(
    ABANDONED_OBJECT_FILL_COLOR, text_color=Color(255, 255, 255), text_thickness=2)

suspicious_object_text_annotator = TextAnnotator(
    SUSPICIOUS_OBJECT_FILL_COLOR, text_color=Color(255, 255, 255), text_thickness=2)


regular_object_marker_annotator = MarkerAnntator(
    color = REGUAL_OBJECT_FILL_COLOR
)

abandoned_object_marker_annotator = MarkerAnntator(
    color = ABANDONED_OBJECT_FILL_COLOR
)

suspicious_object_marker_annotator = MarkerAnntator(
    color = SUSPICIOUS_OBJECT_FILL_COLOR
)

person_marker_annotator = MarkerAnntator(
    color = PERSON_FILL_COLOR
)

regular_object_visualization_annotator = ObjectAnnotator(
    color = REGUAL_OBJECT_FILL_COLOR
)

abandoned_object_visualization_annotator = ObjectAnnotator(
    color = ABANDONED_OBJECT_FILL_COLOR
)

suspicious_object_visualization_annotator = ObjectAnnotator(
    color = SUSPICIOUS_OBJECT_FILL_COLOR
)

##ALG

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!rm -rf /content/Results
!rm -rf /content/MOTA

In [ ]:
if not os.path.isdir("/content/MOTA"):
    os.mkdir("/content/MOTA")

for yolo_version in ["YOLOv5", "YOLOv8"]:
    if yolo_version == "YOLOv5":
        model = torch.hub.load('ultralytics/yolov5', 'custom', f'{dir_of_test}/YOLOv5.pt', device=0)
    if yolo_version == "YOLOv8":
        model = YOLO(f'{dir_of_test}/YOLOv8.pt')
        model.fuse()

    model.agnostic = False  # NMS class-agnostic
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    model.names

    for tracking_method in ['BYTETracker', 'StrongSORT', 'SORT']:

        if tracking_method == "BYTETracker":
            tracker = BYTETracker(BYTETrackerArgs())
        if tracking_method == "StrongSORT":
            tracker = StrongSORT(
                model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
                device='cuda:0',
                max_age=300,
                fp16=False,
            )
        if tracking_method == "SORT":
            tracker = sort.SORT(max_age=5, min_hits=3, iou_threshold=0.4)

        for method_of_ab_ch in ['I', 'II', 'III']:

            name_of_dir = "/content/MOTA/"+video_name+"_"+yolo_version+"_"+tracking_method+"_"+method_of_ab_ch

            if not os.path.isdir(name_of_dir):
                os.mkdir(name_of_dir)

            for conf_of_yolo in ['CONF=0.3', 'CONF=0.6', 'CONF=0.9']:
                if conf_of_yolo == "CONF=0.3":
                    model.conf = 0.3
                if conf_of_yolo == 'CONF=0.6':
                    model.conf = 0.6
                if conf_of_yolo == 'CONF=0.9':
                    model.conf = 0.9

                for iou_of_yolo in ["IOU=0.45", "IOU=0.6", "IOU=0.75"]:
                    if iou_of_yolo == "IOU=0.45":
                        model.iou = 0.45
                    if iou_of_yolo == "IOU=0.6":
                        model.iou = 0.6
                    if iou_of_yolo == "IOU=0.75":
                        model.iou = 0.75

                    TARGET_VIDEO_PATH = f"{HOME}/Results/{video_name}_{yolo_version}_{tracking_method}_{method_of_ab_ch}_{conf_of_yolo}_{iou_of_yolo}.mp4"

                    name_of_file_with_dets = name_of_dir+"/"+yolo_version+"_"+tracking_method+"_"+method_of_ab_ch+"_"+conf_of_yolo+"_"+iou_of_yolo+".txt"
                    name_of_file_with_abandoned = name_of_dir+"/"+yolo_version+"_"+tracking_method+"_"+method_of_ab_ch+"_"+conf_of_yolo+"_"+iou_of_yolo+"_"+"ab_ob"+".txt"

                    if os.path.exists(name_of_file_with_dets):
                        print("SKIP")
                        continue
                    else:
                        create_or_empty_file(name_of_file_with_dets)
                        print(name_of_file_with_dets)

                    if os.path.exists(name_of_file_with_abandoned):
                        print("SKIP")
                    else:
                        create_or_empty_file(name_of_file_with_abandoned)
                        print(name_of_file_with_abandoned)


                    # initiate video writer
                    video_config = VideoConfig(
                        fps=fps_of_source,
                        width=width_of_source,
                        height=height_of_source)

                    video_writer = get_video_writer(
                        target_video_path=TARGET_VIDEO_PATH,
                        video_config=video_config)
                    # get fresh video frame generator
                    frame_iterator = iter(generate_frames(video_file=SOURCE_VIDEO_PATH))

                    arr_of_abandoned_objects = np.empty((0,4), dtype=np.object)

                    ABANDONED_CHECK=0
                    # loop over frames
                    frame_number=0
                    # for frame in frame_iterator:
                    for frame in tqdm(frame_iterator, total=length_of_source):
                        frame_number=frame_number+1
                        # run detector
                        results = model(frame)
                        if yolo_version=="YOLOv5":
                            detections = Detection.from_results(
                                pred=Detections_from_YOLOv5(results),
                                names=model.names)
                        if yolo_version=="YOLOv8":
                            detections = Detection.from_results(
                                pred=Detections_from_YOLOv8(results),
                                names=model.names)
                        if detections != []:
                        # Copy of frame for annotations
                            annotated_image = frame.copy()
                            # run ByteTracker
                            if tracking_method=="BYTETracker":
                                tracks = tracker.update(
                                        output_results=detections2boxes_BYTE(detections=detections),
                                        img_info=frame.shape,
                                        img_size=frame.shape
                                    )
                            # run StrongSORT
                            if tracking_method=="StrongSORT":
                                if yolo_version=="YOLOv5":
                                    tracks = tracker.update(Detections_from_YOLOv5(results), frame)
                                if yolo_version=="YOLOv8":
                                    tracks = tracker.update(Detections_from_YOLOv8(results), frame)
                            # run SORT
                            if tracking_method=="SORT":
                                tracker.run(detections2boxes_SORT(detections, False), 0)
                                tracks = tracker.get_tracks(0)

                            if tracks!=[]:
                                if tracking_method=="BYTETracker":
                                    # assigns the ID of tracks to detection
                                    tracked_detections = match_detections_with_tracks_BYTE(detections=(detections), tracks=tracks)

                                if tracking_method=="StrongSORT":
                                    # assigns the ID of tracks to detection
                                    tracked_detections = match_detections_with_tracks_STRONG(detections=(detections), tracks=tracks)
                                if tracking_method=="SORT":
                                    tracked_detections= match_detections_with_tracks_SORT(detections=(detections), tracks=tracks)

                                # Filter detections by class
                                tracked_backpack_detections = filter_detections_by_class(detections=tracked_detections, class_name="backpack")
                                tracked_handbag_detections = filter_detections_by_class(detections=tracked_detections, class_name="handbag")
                                tracked_person_detections = filter_detections_by_class(detections=tracked_detections, class_name="person")
                                tracked_suitcase_detections = filter_detections_by_class(detections=tracked_detections, class_name="suitcase")

                                tracked_object_detections= tracked_backpack_detections + tracked_handbag_detections + tracked_suitcase_detections
                                # calculate regular objects
                                if method_of_ab_ch=="I":
                                    regular_object_detections = get_regular_object(
                                        tracked_person_detections,
                                        tracked_object_detections
                                    )

                                if method_of_ab_ch=="II":
                                    regular_object_detections = get_regular_object_fourth(
                                        tracked_person_detections,
                                        tracked_object_detections
                                    )

                                if method_of_ab_ch=="III":
                                    regular_object_detections = get_regular_object_fifth(
                                        tracked_person_detections,
                                        tracked_object_detections
                                    )



                                unique_regular_object_detections=[]

                                for x in regular_object_detections:
                                    if x not in unique_regular_object_detections:
                                        unique_regular_object_detections.append(x)

                                # calculate abandoned objects
                                abandoned_objects_detections = [x for x in tracked_object_detections if x not in unique_regular_object_detections]

                                unique_abandoned_objects=[]

                                for x in abandoned_objects_detections:
                                    if x not in unique_abandoned_objects:
                                        unique_abandoned_objects.append(x)


                                arr_of_abandoned_objects=classification_of_abandoned_objects(arr_of_abandoned_objects, unique_abandoned_objects, frame_number)

                                for_remove = []

                                for i in range(len(arr_of_abandoned_objects)):
                                    arr_box = detections2boxes_for_single_detection(arr_of_abandoned_objects[i][0])
                                    for j in range(len(unique_regular_object_detections)):
                                        det_box = detections2boxes_for_single_detection(unique_regular_object_detections[j])
                                        if arr_of_abandoned_objects[i][1] == True and  box_iou_batch(arr_box, det_box) > 0.6 and frame_number - arr_of_abandoned_objects[i][3] < 100:
                                            for_remove.append(j)

                                for_remove = sorted(for_remove, reverse=True)
                                for idx in for_remove:
                                    if idx < len(unique_regular_object_detections):
                                        unique_regular_object_detections.pop(idx)

                                detections_MOT=tracked_person_detections+unique_regular_object_detections+unique_abandoned_objects
                                mot_txt(name_of_file_with_dets, frame_number, detections_MOT)

                                # annotate video frame
                                for i in range(len(arr_of_abandoned_objects)):
                                    if arr_of_abandoned_objects[i][1] == True and frame_number - arr_of_abandoned_objects[i][3] < 100:
                                        ABANDONED_CHECK=1
                                        annotated_image = abandoned_object_text_annotator.annotate(
                                            image=annotated_image,
                                            detections=[arr_of_abandoned_objects[i][0]])
                                        annotated_image = abandoned_object_marker_annotator.annotate(
                                            image=annotated_image,
                                            detections=[arr_of_abandoned_objects[i][0]])

                                    if arr_of_abandoned_objects[i][1] == False and frame_number == arr_of_abandoned_objects[i][3]:
                                        annotated_image = suspicious_object_text_annotator.annotate(
                                            image=annotated_image,
                                            detections=[arr_of_abandoned_objects[i][0]])
                                        annotated_image = suspicious_object_marker_annotator.annotate(
                                            image=annotated_image,
                                            detections=[arr_of_abandoned_objects[i][0]])

                                if ABANDONED_CHECK == 1:
                                    ABANDONED_CHECK=0
                                    with open(name_of_file_with_abandoned, 'r+') as f:
                                        f.seek(0, 2)  # перемещение курсора в конец файла
                                        f.write(f'{frame_number},')  # собственно, запись

                                annotated_image = person_text_annotator.annotate(
                                    image=annotated_image,
                                    detections=tracked_person_detections)

                                annotated_image = regular_object_text_annotator.annotate(
                                    image=annotated_image,
                                    detections=unique_regular_object_detections)


                                annotated_image = person_marker_annotator.annotate(
                                    image=annotated_image,
                                    detections=tracked_person_detections)

                                annotated_image = regular_object_marker_annotator.annotate(
                                    image=annotated_image,
                                    detections=unique_regular_object_detections)

                            # save video frame
                            video_writer.write(annotated_image)

                        else:
                            video_writer.write(frame)

                    video_writer.release()

In [ ]:
!zip -r MOTA1.zip /content/MOTA

In [ ]:
!cp MOTA1.zip "drive/My Drive/"

#END